In [40]:
import random
import matplotlib.pyplot as plt

In [70]:
class TournamentPopulation(object):
    def __init__(self, individual_class, population_size=100, infection_rate=0.5, mutation_prob=0.02, deme_size=5):
        self.infection_rate = infection_rate
        self.mutation_prob = mutation_prob
        self.deme_size = deme_size
        
        #Build a randomly generated population
        self.population = [individual_class() for i in range(population_size)]
        
        #Get the highest performing individual
        self.highest_fitness_individual = sorted(self.population, key=lambda individual: individual.fitness())[-1]
        self.highest_fitness = self.highest_fitness_individual.fitness()
        
        
    #Tournament selection with two parents, and microbial infection based reproduction
    def reproduction(self, parents):
        parents = sorted(parents, key=lambda individual: individual.fitness())
        winner = parents[1]
        loser = parents[0]
        
        #'infect' the loser of the tournament, the winner should be championed through without changes
        loser.infect(winner, self.infection_rate)
        loser.mutate(self.mutation_prob)
            
        #Check if fitness of new individual is the highest so far
        new_individual_fitness = loser.fitness()
        if self.highest_fitness < new_individual_fitness:
            self.highest_fitness = new_individual_fitness
            self.highest_fitness_individual = loser
            
    #Pick 2 indices of population randomly, with trivial geography
    def selection(self):
        #Select the first parent randomly from the entire population
        first_parent_index = random.randint(0, len(self.population)-1)
        first_parent = self.population[first_parent_index]
        
        #Select the second parent from a deme or subset of the population starting after the first parent
        second_parent_offset = random.randint(0, self.deme_size)
        second_parent_index = (first_parent_index + second_parent_offset - 1) % len(self.population)
        second_parent = self.population[second_parent_index]
        
        return [first_parent, second_parent]
    
    
    def run_cycle(self):
        parents = self.selection()
        self.reproduction(parents)
            


In [71]:
#TODO?: class AbstractBitArrayIndividual(object):

class AbstractLinkedListIndividual(object):
    #Infect this individual with parts of the genotype of another individual
    def infect(self, infector, infection_perc):
        if len(self.genotype) != len(infector.genotype):
            raise ValueError("The genotype of the infecting individual should be the same size")
        
        #Determine how many indices are replaced the infecting 
        infected_indices_count = round(len(self.genotype) * infection_perc)
        
        #Get the indices that are being replaced
        infected_indices = random.sample(range(0, len(self.genotype)), infected_indices_count)
        
        #Replace the infected indexs with the same indexes from the infecting individual
        for infected_index in infected_indices:
            self.genotype[infected_index] = infector.genotype[infected_index]
    
    #How the genotype is encoded and fitness is implementation specific
    def mutate(self, mutation_rate):
        pass
    
    def fitness(self):
        pass
            
class NumberIndividual(AbstractLinkedListIndividual):
    def __init__(self, number_count=3):
        #Make genotype of n values between [0.0, 1.0)
        self.genotype = [random.random() for i in range(number_count)]
            
    #Check if each gene in the genotype should be mutated
    def mutate(self, mutation_rate):
        for i in range(0, len(self.genotype)):
            if random.random() <= mutation_rate:
                self.genotype[i] = random.random()
        
    #Fitness is defined as the sum of all numbers this individual contains
    def fitness(self):
        return sum(self.genotype)
    
class BitIndividual(AbstractLinkedListIndividual):
    def __init__(self, bit_count=250):
        self.genotype = [random.randint(0,1) for i in range(bit_count)]
        
    def mutate(self, mutation_rate):
        for i in range(0, len(self.genotype)):
            if random.random() <= mutation_rate:
                self.genotype[i] = random.randint(0,1)
                
    #Fitness is defined as the sum of the first half of the bits 
    #minus the sum of the second half of the bits
    def fitness(self):
        middle_index = int(len(self.genotype) / 2)
        
        first_half_sum = sum(self.genotype[0: middle_index])
        second_half_sum = sum(self.genotype[middle_index: -1])
        
        return first_half_sum - second_half_sum

In [72]:
%matplotlib widget

def makePopulationRunGraph(pop, runs):
    last_high = 0
    x = []
    y = []

    #Only marks the index at which a new best performing individual exists, this works because of free elitism
    for i in range(runs):
        pop.run_cycle()
        if last_high != pop.highest_fitness:
            x.append(i)
            y.append(pop.highest_fitness)

            last_high = pop.highest_fitness
    return (x,y)
    
#Just a quick graph of that marks the best performing individual
numberVals = makePopulationRunGraph(TournamentPopulation(NumberIndividual), 10000)

plt.plot(numberVals[0], numberVals[1], 'ro')
plt.show()

FigureCanvasNbAgg()